<a href="https://colab.research.google.com/github/mkernik/Bbox_assignment/blob/main/RutgersPlaceNames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/MetadataSprint/

/content/drive/My Drive/MetadataSprint


In [ ]:
df = pd.read_csv("RutgersSpatialCoverage.csv")

In [ ]:
place_names = dict(zip(df.geoportal_id, df.SpatialCoverage))
print(place_names)

In [ ]:
df2 = pd.read_csv("NJCountyBBox_simple.csv")

In [ ]:
df2.head()
NJ_counties_dict = dict(zip(df2.CountyName, df2.BoundingBox))
print(NJ_counties_dict)

In [ ]:
for county in NJ_counties_dict.keys():
  print (county)

Atlantic
Bergen
Burlington
Camden
Cape May
Cumberland
Essex
Gloucester
Hudson
Hunterdon
Mercer
Middlesex
Monmouth
Morris
Ocean
Passaic
Salem
Somerset
Sussex
Union
Warren


In [ ]:
df3 = pd.read_csv("NJ_Muni_Bbox_Extents.csv")

In [ ]:
df3.head()
NJ_muni = dict(zip(df3.MUN_LABEL, df3.bbox))
print(NJ_muni)

In [ ]:
df3.head()
NJ_alt_muni = dict(zip(df3.NAME, df3.bbox))
print(NJ_alt_muni)

**Matching municipalities**

In [ ]:
muni_count = {}
#index = 0
for key, value in place_names.items():
  #index += 1
  match = []
  multi_term = value.split("|")
  for term in multi_term:  
    for muni in NJ_muni.keys():
      if muni in term:
        match.append(muni)
  unique = list(set(match))
  if len(unique) == 1:
    for x in match:
      muni_count[key] = [value, len(unique), x, NJ_muni[x]]
  else:
    muni_count[key] = [value, len(unique), "", ""]
print (len(muni_count))

4579


In [ ]:
muni_df = pd.DataFrame.from_dict(muni_count, orient = "index")
muni_df.to_csv('MatchingMunicipalities.csv')
files.download("MatchingMunicipalities.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Matching Counties**

In [ ]:
count = {}
#index = 0
for key, value in place_names.items():
  #index += 1
  match = []
  multi_term = value.split("|")
  for term in multi_term:
    for county in NJ_counties_dict.keys():
      if (county + " County") in term:
      #if county in term:
        match.append(county)
  unique = list(set(match))
  if len(unique) == 1:
    for x in match:
      count[key] = [value, len(unique), x, NJ_counties_dict[x]]
  else:
    count[key] = [value, len(unique), "", ""]
print (len(count))

4579


In [ ]:
new_df = pd.DataFrame.from_dict(count, orient = "index")
new_df.to_csv('MatchingCounties.csv')
files.download("MatchingCounties.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Combining matching municipalities and counties**

In [ ]:
count = {}
for key, value in place_names.items():
  muni_match = []
  multi_term = value.split("|")
  for term in multi_term:  
    for muni in NJ_muni.keys():
      if muni in term:
        muni_match.append(muni)
  muni_unique = list(set(muni_match))
  if len(muni_unique) == 1:
    for x in muni_match:
      count[key] = [value, len(muni_unique), x, NJ_muni[x],"","",""]
  else:
    county_match = []
    for term in multi_term:
      for county in NJ_counties_dict.keys():
        if (county + " County") in term:
          county_match.append(county)
    county_unique = list(set(county_match))
    if len(county_unique) == 1:
      for y in county_match:
        count[key] = [value, len(muni_unique), "", "", len(county_unique), y, NJ_counties_dict[y]]
    else:
      count[key] = [value, len(muni_unique), "", "", len(county_unique), "", ""]
print (len(count))


4579


**Additional attempts to match municipalities** 

In [ ]:
count = {}
alt_count ={}
unmatched = {}
for key, value in place_names.items():
  muni_match = []
  multi_term = value.split("|")
  for term in multi_term:  
    for muni in NJ_muni.keys():
      if muni in term:
        muni_match.append(muni)
  muni_unique = list(set(muni_match))
  if len(muni_unique) == 1:
    for x in muni_match:
      count[key] = [value, "Municipal", x, NJ_muni[x]]
  else:
    county_match = []
    for term in multi_term:
      for county in NJ_counties_dict.keys():
        if (county + " County") in term:
          county_match.append(county)
    county_unique = list(set(county_match))
    if len(county_unique) == 1:
      for y in county_match:
        count[key] = [value, "County", y, NJ_counties_dict[y]]
    else:
      alt_muni_match = []
      for term in multi_term:  
        for alt_muni in NJ_alt_muni.keys():
          if (alt_muni + ", New Jersey") in term:
            alt_muni_match.append(alt_muni)
          else:
            alt_muni_not = alt_muni.replace(" Township", "")
            alt_muni_nob = alt_muni_not.replace(" Borough", "")
            if (alt_muni_nob + ", New Jersey") in term:
              alt_muni_match.append(alt_muni)
      alt_muni_unique = list(set(alt_muni_match))
      if len(alt_muni_unique) == 1:
        if len(county_unique) < 2:
          for z in alt_muni_match:
            alt_count[key] = [value, "Municipal", z, NJ_alt_muni[z]]
      else:
        unmatched[key] = [value, len(muni_unique), len(county_unique)]
print (len(count))
print (len(alt_count))
print (len(unmatched))

3319
303
942


**Code to download resulting CSVs**

In [ ]:
combine_df = pd.DataFrame.from_dict(count, orient = "index")
combine_df.to_csv('MatchingMuniAndCounties.csv')
files.download("MatchingMuniAndCounties.csv")

In [ ]:
print(len(unmatched))
unmatched_df = pd.DataFrame.from_dict(unmatched, orient = "index")
unmatched_df.to_csv('UnMatchedPlaces.csv')
files.download("UnMatchedPlaces.csv")

In [ ]:
alt_matched_df = pd.DataFrame.from_dict(alt_count, orient = "index")
alt_matched_df.to_csv('Alternative_MatchedPlaces.csv')
files.download("Alternative_MatchedPlaces.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Work still to do:
*   Look at Kalan's process for matching with Geonames
*   Look for more opportunities with unmatched places and better matching for records that only matched with county


Total records: 4579

*   1257 don't have "[Name] County"  (27%)
*   3168 have only one "[Name] County" (69%)
*   154 has more than one "[Name] County" (3%)

---

*   515 don't have one of the named counties in their keyword list  (11%)
*   3658 have only one of the named counties in their keyword list  (80%)
*   406 have more than one of the named counties in their keyword list (9%)



---
combined county/muni 9/29/2022 end of day

1137 no match muni or county (24%)
789 have only one match MUNI_LABEL (17%)
2530 have only one "[Name] County" (55%)
123 match with more than one "[Name] County" (2-3%)